# <span style='color:#15317E'>02 Capstone Project Indicators</span>

In [1]:
######
#### Author: Byron Stuart
#### Data Science Immersive Capstone Project
#### Date 06 June 2017
######

## <span style='color:#7D6115'>Capstone Project Navigation</span>
<a href='./01%20capstone_setup.ipynb'>Step 1 - Capstone Setup Notebook</a><br>
Step 2 - Capstone Project Indicators<br>
<a href='./03%20capstone_eda_and_models.ipynb'>Step 3 - Capstone EDA Notebook</a><br>
<a href='./04%20capstone_report.ipynb'>Step 4 - Capstone Report Notebook</a><br>

## <span style='color:#7D6115'>Capstone Indicator Setup</span>
For the initial project setup, EDA, SQL database and analysis, see the capstone setup notebook. All the data needed by this notebook is read in from the SQL database.

### Imports
Import widely used packages and setup dataframes from the SQL database.

In [2]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra

# Import Sqlite3 Library
import sqlite3
#The SQL Sub-library from Pandas will allow us to run SQL queries within python.
from pandas.io import sql

# Establishing the Connection to our Database.  If no database exists here, this will create one.
connection = sqlite3.connect('../input/data.db.sqlite')

# define function to use when querying the sql database
def Q(query, db=connection):
    return sql.read_sql(query, db)

data = pd.DataFrame(Q('select * from world_health'))

world_data = pd.DataFrame(Q('select * from world_health_world'))
income_data = pd.DataFrame(Q('select * from world_health_income'))
combos_data = pd.DataFrame(Q('select * from world_health_combos'))
single_data = pd.DataFrame(Q('select * from world_health_single'))

indicators_df = pd.DataFrame(Q('select * from world_health_indicators'))

### Retrieve all the indicators from the database

In [3]:
indicators = data['Indicator Name'].unique()

income_indicators = list(indicators_df[indicators_df.name=='income'].indicator)
combos_indicators = list(indicators_df[indicators_df.name=='combos'].indicator)
single_indicators = list(indicators_df[indicators_df.name=='single'].indicator)

### Look at all the indicators

In [4]:
# Original number of indicators: 345
indicators = combos_indicators
print("Number of indicators:", len(combos_indicators), "\n")
for ind in combos_indicators:
    print(ind)

Number of indicators: 345 

age population, age 16, female, interpolated
age population, age 10, female, interpolated
primary completion rate, total (% of relevant age group)
age population, age 12, male, interpolated
age population, age 15, male, interpolated
population ages 80 and older, male (% of male population)
share of women employed in the nonagricultural sector (% of total nonagricultural employment)
school enrollment, secondary, female (% gross)
population ages 65 and above, male
condom use, population ages 15-24, male (% of males ages 15-24)
demand for family planning satisfied by any methods (% of married women with demand for family planning)
school enrollment, primary, female (% net)
population ages 00-14, male (% of total)
age population, age 07, female, interpolated
school enrollment, primary, female (% gross)
school enrollment, primary (% gross)
age population, age 17, female, interpolated
age population, age 06, male, interpolated
literacy rate, youth male (% of males

### Choose grouped features to be removed
Some features will be removed as they are subsets of a broader grouping.

In [5]:
combos_indicators.remove('population, total') # have male and female
combos_indicators.remove('improved sanitation facilities (% of population with access)') # have urban and rural
combos_indicators.remove('improved water source (% of population with access)') # have urban and rural
combos_indicators.remove('unemployment, total (% of total labor force)') # have male and female
combos_indicators.remove('health expenditure, total (% of gdp)') # have public and private
combos_indicators.remove('health expenditure per capita (current us$)') # have ppp
combos_indicators.remove('school enrollment, secondary (% net)') # have male and female
combos_indicators.remove('school enrollment, secondary (% gross)') # have male and female
combos_indicators.remove('school enrollment, primary (% net)') # have male and female
combos_indicators.remove('school enrollment, primary (% gross)') # have male and female
combos_indicators.remove('school enrollment, tertiary (% gross)') # have male and female
combos_indicators.remove('primary completion rate, total (% of relevant age group)') # have male and female

### Function to group indicators

In [6]:
# Function that has 2 parameters, a list of indicators and a string to search for in those indicators.
# After the indicators matching 'identity' have been found remove them from indicators and return the results,
# this will make sure the same indicator is not included in a subsequent list.
def population_group(indicators, identity):
    identity = identity.lower()
    population = [ind for ind in indicators if identity in ind]
    print("Related to", identity, len(population))
    indicators = np.setdiff1d(indicators, population)
    return population, indicators

### Attempt to group indicators into related categories
Not being used at this stage of the project, some groups come up with 0 matches as features have already been removed due to null values.

In [7]:
# Categorise indicators into groups
life_expectancy, indicators = population_group(indicators, 'Life expectancy')
diabetes, indicators = population_group(indicators, 'diabetes')
undernourished, indicators = population_group(indicators, 'undernourished')
hiv, indicators = population_group(indicators, 'HIV')
aids, indicators = population_group(indicators, 'aids estimated deaths')
age_population, indicators = population_group(indicators, 'Age population')
population_ages, indicators = population_group(indicators, 'Population ages')
female_population, indicators = population_group(indicators, 'Female population')
male_population, indicators = population_group(indicators, 'Male population')
health_expenditure, indicators = population_group(indicators, 'Health expenditure')
mortality, indicators = population_group(indicators, 'Mortality')
survival_65, indicators = population_group(indicators, 'survival to age 65')
school_enrollment, indicators = population_group(indicators, 'School enrollment')

# some indicators with this 'identity' still show up down near the bottom as being unclassified
# could be a bug with the function setdiff1d, might only be Windows that's affected?
primary_completion_rate = population_group(indicators, 'Primary completion rate')

malnutrition, indicators = population_group(indicators, 'Malnutrition')
overweight, indicators = population_group(indicators, 'overweight')
immunization, indicators = population_group(indicators, 'Immunization')
sanitation, indicators = population_group(indicators, 'sanitation')
wasting, indicators = population_group(indicators, 'wasting')
birth, indicators = population_group(indicators, 'birth')
death, indicators = population_group(indicators, 'death')
condom, indicators = population_group(indicators, 'Condom')
contraceptive, indicators = population_group(indicators, 'contraceptive')
water, indicators = population_group(indicators, 'water')
population, indicators = population_group(indicators, 'Population')
anemia, indicators = population_group(indicators, 'anemia')
smoking, indicators = population_group(indicators, 'Smoking')
diarrhea, indicators = population_group(indicators, 'Diarrhea')
unemployment, indicators = population_group(indicators, 'Unemployment')
maternal, indicators = population_group(indicators, 'Maternal')
marriage, indicators = population_group(indicators, 'marriage')
tuberculosis, indicators = population_group(indicators, 'Tuberculosis')

Related to life expectancy 3
Related to diabetes 1
Related to undernourished 1
Related to hiv 18
Related to aids estimated deaths 1
Related to age population 53
Related to population ages 54
Related to female population 17
Related to male population 17
Related to health expenditure 9
Related to mortality 12
Related to survival to age 65 2
Related to school enrollment 9
Related to primary completion rate 2
Related to malnutrition 6
Related to overweight 6
Related to immunization 6
Related to sanitation 2
Related to wasting 6
Related to birth 10
Related to death 12
Related to condom 2
Related to contraceptive 2
Related to water 2
Related to population 22
Related to anemia 4
Related to smoking 2
Related to diarrhea 2
Related to unemployment 2
Related to maternal 1
Related to marriage 2
Related to tuberculosis 3


### Show number of indicators that will be used

In [8]:
len(combos_indicators)

333

### Remove age and demographic indicators <a name='age_indicators' />

In [9]:
combos_indicators = list(set(combos_indicators) - set(population_ages) - \
                         set(female_population) - set(male_population) -\
                         set(mortality) - set(death) - set(survival_65) - \
                         set(['population, female']) - set(['population, male']))

### 116 indicators removed

In [10]:
len(combos_indicators)

217

In [11]:
combos_indicators

['health expenditure per capita, ppp',
 'school enrollment, secondary, male (% gross)',
 'malnutrition prevalence, weight for age, male (% of children under 5)',
 'age population, age 23, male, interpolated',
 'net migration',
 'age population, age 03, male, interpolated',
 'immunization, hepb3 (% of one-year-old children)',
 'age population, age 04, male, interpolated',
 'prevalence of anemia among women of reproductive age (% of women ages 15-49)',
 'wanted fertility rate (births per woman)',
 'age population, age 12, female, interpolated',
 'age population, age 02, female, interpolated',
 'children (ages 0-14) newly infected with hiv',
 'pregnant women receiving prenatal care (%)',
 'improved water source, rural (% of rural population with access)',
 'incidence of hiv (% of uninfected population ages 15-49)',
 'prevalence of overweight (% of children under 5)',
 'sex ratio at birth (male births per female births)',
 'smoking prevalence, males (% of adults)',
 'age population, age 0,

### Write life_expectancy and combos_indicators to a csv file for other notebooks

In [12]:
life_expectancy_df = pd.DataFrame(life_expectancy)
life_expectancy_df.rename(columns={0:'indicator'}, inplace=True)
life_expectancy_df.to_csv('./life_expectancy_df.csv')

combos_indicators_df = pd.DataFrame(combos_indicators)
combos_indicators_df.rename(columns={0:'indicator'}, inplace=True)
combos_indicators_df.to_csv('./combos_indicators_df.csv')

### Close the SQL connection

In [13]:
connection.close()